In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('D:\\Data Science\\Code basics\\py-master\\ML\\14_naive_bayes\\titanic.csv')

In [3]:
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [4]:
## basic data exploration, drop some irrelevent variables
df.drop(['PassengerId','Name','Parch','SibSp','Ticket','Cabin','Embarked'], axis='columns', inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [5]:
## We have target variable, we have to separate it
target = df.Survived
inputs = df.drop('Survived', axis = 'columns')

In [6]:
## converting the sex columns into dummies
dummies = pd.get_dummies(inputs.Sex)
dummies.head()

,female,male
0,False,True
1,True,False
2,True,False
3,True,False
4,False,True


In [7]:
## Addending the dummies to the inputs
inputs = pd.concat([inputs, dummies], axis='columns')
inputs.head()

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,False,True
1,1,female,38.0,71.2833,True,False
2,3,female,26.0,7.9250,True,False
3,1,female,35.0,53.1000,True,False
4,3,male,35.0,8.0500,False,True


In [8]:
## drop the original Sex column
inputs.drop('Sex', axis='columns', inplace=True)
inputs.head()
# now we have only numeric details not the texts

,Pclass,Age,Fare,female,male
0,3,22.0,7.2500,False,True
1,1,38.0,71.2833,True,False
2,3,26.0,7.9250,True,False
3,1,35.0,53.1000,True,False
4,3,35.0,8.0500,False,True


In [9]:
## Checking null values
inputs.columns[inputs.isna().any()]
## We have NA values in the Age column

Index(['Age'], dtype='object')

In [10]:
## checking 1st 10 values
inputs.Age[:10]
## we have a NA

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [11]:
## Filling the NA values using mean of the column
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head(6)

,Pclass,Age,Fare,female,male
0,3,22.000000,7.2500,False,True
1,1,38.000000,71.2833,True,False
2,3,26.000000,7.9250,True,False
3,1,35.000000,53.1000,True,False
4,3,35.000000,8.0500,False,True
5,3,29.699118,8.4583,False,True


In [23]:
## splitting dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs, target,test_size=0.2)

In [24]:
len(X_train), len(X_test)

(712, 179)

In [25]:
## importing the Naive Bayes model we will be using the GaussianNB which is similar to bell curve
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [26]:
model.fit(X_train,y_train)

GaussianNB()

In [27]:
model.score(X_test, y_test) ## we got 77% of the accuracy

0.7541899441340782

In [28]:
## Checking the X_test samples
X_test[:10]

,Pclass,Age,Fare,female,male
763,1,36.000000,120.0000,True,False
870,3,26.000000,7.8958,False,True
455,3,29.000000,7.8958,False,True
259,2,50.000000,26.0000,True,False
503,3,37.000000,9.5875,True,False
125,3,12.000000,11.2417,False,True
843,3,34.500000,6.4375,False,True
693,3,25.000000,7.2250,False,True
17,2,29.699118,13.0000,False,True
202,3,34.000000,6.4958,False,True


In [29]:
y_test[:10]

763    1
870    0
455    1
259    1
503    0
125    1
843    0
693    0
17     1
202    0
Name: Survived, dtype: int64

In [30]:
model.predict(X_test[:10]) ## testing the model for 1st 10 samples, there are some errors!!!

array([1, 0, 0, 1, 1, 0, 0, 0, 0, 0], dtype=int64)

In [31]:
## checking the probability
result = model.predict_proba(X_test[:10]) ## in first row, chance of 0.2% not survived, 99 % survived

In [32]:
import numpy as np
np.set_printoptions(suppress=True)

result * 100

array([[ 0.02024327, 99.97975673],
       [99.01881198,  0.98118802],
       [99.05016417,  0.94983583],
       [ 2.04844856, 97.95155144],
       [ 4.72054596, 95.27945404],
       [98.76847603,  1.23152397],
       [99.08513904,  0.91486096],
       [99.00435401,  0.99564599],
       [97.91833094,  2.08166906],
       [99.08258452,  0.91741548]])

### Calculate using the Cross validation

In [33]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(GaussianNB(),X_train, y_train, cv=10)
np.average(score)

0.7879499217527386